In [79]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

In [80]:
# tool creation

@tool
def multiply(a:int,b:int)->int:
    """Given 2 numbers a and b this tool returns their product"""
    return a*b


In [81]:
multiply.name

'multiply'

In [82]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [83]:
llm=HuggingFaceEndpoint(
    repo_id='deepseek-ai/DeepSeek-V3.2-Exp',
    task="text-generation"
)

model=ChatHuggingFace(llm=llm)

Tool binding

In [84]:
llm_with_tools=model.bind_tools([multiply])

In [85]:
model.invoke("what's 9 multiplied by 2")

AIMessage(content="That's **18**.\n\nHere are a few other ways to think about it:\n\n*   **9 + 9 = 18**\n*   **10 × 2 = 20**, then subtract 2 to get **18**.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 12, 'total_tokens': 60}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--be8231eb-32b8-49ab-a684-f2b27f03a016-0', usage_metadata={'input_tokens': 12, 'output_tokens': 48, 'total_tokens': 60})

In [86]:
result=llm_with_tools.invoke('can you multiply 3 with 10').tool_calls

In [87]:
multiply.invoke({'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'chatcmpl-tool-4279c0a3cba148339ad0ad526cd408da',
  'type': 'tool_call'})

ToolMessage(content='30', name='multiply', tool_call_id='chatcmpl-tool-4279c0a3cba148339ad0ad526cd408da')

In [88]:
multiply.invoke(result[0])

ToolMessage(content='30', name='multiply', tool_call_id='chatcmpl-tool-1a416d988571453c9c49c891129f5db3')

In [89]:
llm_with_tools.invoke("Hi how are you")

AIMessage(content="I'm doing well, thank you for asking! I'm here to help you with various tasks, including mathematical calculations. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 233, 'total_tokens': 265}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--b92cc887-c9a5-4b55-a978-3c7573306ac4-0', usage_metadata={'input_tokens': 233, 'output_tokens': 32, 'total_tokens': 265})

In [90]:
query=HumanMessage("can you multiply 6 with 5")

In [91]:
message=[query]

In [92]:
message

[HumanMessage(content='can you multiply 6 with 5', additional_kwargs={}, response_metadata={})]

In [93]:
result=llm_with_tools.invoke(message)

In [94]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 6, "b": 5}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 237, 'total_tokens': 280}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ec41ebf7-7141-42e4-b454-bdccb47c4ece-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 237, 'output_tokens': 43, 'total_tokens': 280})

In [95]:
message.append(result)

In [96]:
message

[HumanMessage(content='can you multiply 6 with 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 6, "b": 5}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 237, 'total_tokens': 280}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ec41ebf7-7141-42e4-b454-bdccb47c4ece-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 237, 'output_tokens': 43, 'total_tokens': 280})]

In [97]:
tool_result=multiply.invoke(result.tool_calls[0])

In [98]:
tool_result

ToolMessage(content='30', name='multiply', tool_call_id='chatcmpl-tool-4c286cd93309499ebba90ca853701b8c')

In [99]:
message.append(tool_result)

In [100]:
message

[HumanMessage(content='can you multiply 6 with 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 6, "b": 5}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 237, 'total_tokens': 280}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ec41ebf7-7141-42e4-b454-bdccb47c4ece-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'chatcmpl-tool-4c286cd93309499ebba90ca853701b8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 237, 'output_tokens': 43, 'total_tokens': 280}),
 ToolMessage(content='30', name='multiply', tool_call_id='chatcmpl-tool-4c286cd93309499ebba90ca853701b8c')]

In [101]:
llm_with_tools.invoke(message)

AIMessage(content='The product of 6 multiplied by 5 is 30.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 300, 'total_tokens': 314}, 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--2949fe86-0fd3-459e-a5fd-9c1fad8459bb-0', usage_metadata={'input_tokens': 300, 'output_tokens': 14, 'total_tokens': 314})

Currency converter 

In [102]:
# tools creation

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str,target_currency: str) -> float:
    """
    this funcion fetches the currency conversion factor between a given base currency and a target currency
    """
    url=f"https://v6.exchangerate-api.com/v6/049e041553fe3239ad14498e/pair/{base_currency}/{target_currency}"
    response=requests.get(url)
    return response.json()
@tool
def convert(base_currency_value: int,conversion_rate: Annotated[float,InjectedToolArg])->float:
    """
    Given a currency conversion rate this function calculates the target currency value form a given base currency value
    
    """
    return base_currency_value*conversion_rate


In [103]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1761004802,
 'time_last_update_utc': 'Tue, 21 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1761091202,
 'time_next_update_utc': 'Wed, 22 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.9348}

In [104]:
convert.invoke({'base_currency_value':10,'conversion_rate':87.9348})

879.348

In [124]:
llm=HuggingFaceEndpoint(
    repo_id='deepseek-ai/DeepSeek-V3.2-Exp',
    task="text-generation"
)
model=ChatHuggingFace(llm=llm)

In [120]:
llm_with_tools=model.bind_tools([get_conversion_factor,convert])

In [128]:
messages=[HumanMessage("What is the covnersion factor between USD and INR,"),HumanMessage("and based on conversion factor can you convert 10 USD to INR")]

In [129]:
messages

[HumanMessage(content='What is the covnersion factor between USD and INR,', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and based on conversion factor can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [130]:
ai_message=llm_with_tools.invoke(messages)

In [133]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'fc_4b112073-0f5b-4aea-bdc5-3d8bb7f34de6',
  'type': 'tool_call'}]

In [134]:
ai_message = llm_with_tools.invoke(messages)
print(ai_message.tool_calls)

# Extract first tool call
tool_call = ai_message.tool_calls[0]
if tool_call["name"] == "get_conversion_factor":
    response = get_conversion_factor.invoke(tool_call["args"])
    conversion_rate = response['conversion_rate'] if 'conversion_rate' in response else response['conversion_rate']  # adapt to your JSON
    result = convert.invoke({'base_currency_value': 10, 'conversion_rate': conversion_rate})
    print(result)


[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'fc_ce46d576-74bb-4709-ba89-1050c9dd6d25', 'type': 'tool_call'}]
879.348
